1. Загружаем все необходимые библиотеки

In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer, QuantileTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, r2_score
import mlflow
from mlflow.models import infer_signature
import optuna
from mlxtend.feature_selection import SequentialFeatureSelector

/home/smash-up/iis_labs/IIS_lab1/.venv_lab1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Загружаем обработанную и очищенную выборку, которая была получена в результате выполнения ЛР1

In [4]:
with open("../data/clean_dataset.pkl", "rb") as f:
    data = pickle.load(f)

df = data['train'].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1998 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1998 non-null   int64  
 1   blue           1998 non-null   int64  
 2   clock_speed    1998 non-null   float64
 3   dual_sim       1998 non-null   int64  
 4   fc             1998 non-null   int64  
 5   four_g         1998 non-null   int64  
 6   int_memory     1998 non-null   int64  
 7   m_dep          1998 non-null   float64
 8   mobile_wt      1998 non-null   int64  
 9   n_cores        1998 non-null   int64  
 10  pc             1998 non-null   int64  
 11  px_height      1998 non-null   int64  
 12  px_width       1998 non-null   int64  
 13  ram            1998 non-null   int64  
 14  sc_h           1998 non-null   int64  
 15  sc_w           1998 non-null   int64  
 16  talk_time      1998 non-null   int64  
 17  three_g        1998 non-null   int64  
 18  touch_screen 

Разделяем датасет на две выборки: обучающую и тестовую в соотношении 25% и 75%

In [5]:
X = df.drop('price_range', axis=1)
y = df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75)

3. Создаём переменные, содержащие названия столбцов с числовыми и категориальными признаками

In [12]:
binary_features = ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
numeric_features = [col for col in X_test.columns 
                    if col not in binary_features + ['price_range']]
print(f"Бинарные (категориальные) признаки: {binary_features}")
print(f"Числовые признаки: {numeric_features}")

Бинарные (категориальные) признаки: ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
Числовые признаки: ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'screen_area', 'pixel_density', 'total_memory', 'is_high_end']


4. Создаём pipeline обработки признаков и обучения модели

In [13]:
s_scaler = StandardScaler()
l_encoder = TargetEncoder()
regressor = RandomForestRegressor(random_state=42)

Преобразования для числовых и категориальных признаков

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, numeric_features),  
        ('bin', l_encoder, binary_features), 
    ],
    remainder='drop' )

5. Обучение baseline-модели и получение метрик качества на тестовой выборке

In [15]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor)])

pipeline.fit(X_train, y_train)

predictions = pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)

{'MAE': 0.17606, 'MSE': 0.0731378, 'R2': 0.9424082788423984}


6. Настройка доступа для подключения к mlflow

In [16]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

mlflow.set_tracking_uri(tracking_uri)   
mlflow.set_registry_uri(registry_uri)

7. Логируем baseline-модель в новый эксперимент и регистрируем модель

In [17]:
EXPERIMENT_NAME = "Mobile Classification 1.0"
RUN_NAME = "baseline model"
REGISTRY_MODEL_NAME = "price_range_prediction_model"

signature =  infer_signature(model_input = X_train.head(5))
input_example = X_train.head(5)

req_file = '../requirements.txt'
params_dict = pipeline.get_params()

/home/smash-up/iis_labs/IIS_lab1/.venv_lab1/lib/python3.12/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Начинаем эксперимент с показателем random_state=42

In [18]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
  
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)

    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/12/23 19:55:19 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/12/23 19:55:19 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc

Начинаем эксперимент с показателем random_state=43

In [19]:
regressor2 = RandomForestRegressor(random_state=43)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor2)])

print(pipeline.fit(X_train, y_train))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['battery_power',
                                                   'clock_speed', 'fc',
                                                   'int_memory', 'm_dep',
                                                   'mobile_wt', 'n_cores', 'pc',
                                                   'px_height', 'px_width',
                                                   'ram', 'sc_h', 'sc_w',
                                                   'talk_time', 'screen_area',
                                                   'pixel_density',
                                                   'total_memory',
                                                   'is_high_end']),
                                                 ('bin', TargetEncoder(),
                                                  ['blue', 'dual_sim', 'four_g',
          

Значение метрик для random_state=43

In [20]:
predictions = pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)

{'MAE': 0.17548000000000002, 'MSE': 0.0712064, 'R2': 0.9439291428859408}


8. С использованием библиотеки sklearn создаём дополнительные признаки, обучаем модель и логируем её

In [21]:
X_train_fe_sklearn = X_train.copy()

pf = PolynomialFeatures(degree=2)

X_train_fe_sklearn

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,screen_area,pixel_density,total_memory,is_high_end
1553,891,0,1.9,1,1,1,18,0.1,153,6,...,17,16,4,1,1,1,272,211.029412,1321,0
1632,1362,1,2.9,0,0,0,47,0.5,186,6,...,19,17,5,0,1,1,323,1935.603715,3676,0
1212,864,0,2.5,0,10,1,24,0.7,148,4,...,13,8,12,1,1,0,104,18046.500000,3988,0
453,1289,0,0.9,0,10,1,25,1.0,178,6,...,6,2,11,1,1,0,12,57311.250000,1068,0
488,1949,0,1.8,0,0,0,20,0.8,186,5,...,17,10,9,0,0,0,170,1803.858824,1145,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648,645,0,2.5,0,10,1,63,1.0,174,4,...,18,8,13,1,0,1,144,1127.444444,3678,0
1802,1289,0,0.8,1,3,1,16,0.9,161,6,...,14,8,20,1,1,0,112,11802.696429,3183,0
701,1717,1,1.3,0,1,0,62,0.7,130,1,...,6,4,5,0,1,0,24,20553.750000,3288,1
995,1456,0,1.6,1,5,0,49,0.2,193,3,...,12,11,16,1,1,0,132,13891.628788,3673,0


In [22]:
pf = PolynomialFeatures(degree=2)
qt = QuantileTransformer()
sp = SplineTransformer(n_knots=3, degree=3)

pf_pipeline = Pipeline(steps=[
    ('poly', pf),
    ('scale', StandardScaler())
])

Преобразуем числовые и категориальные признаки

In [23]:
preprocessor_sklearn = ColumnTransformer(
    transformers=[
        ('num', s_scaler, numeric_features),  
        ('cat', l_encoder, binary_features), 
        ('quantile', qt, numeric_features),
        ('poly', pf_pipeline, ['ram', 'battery_power']),
        ('spline', sp, ['ram'])
    ],
    remainder='drop',
    )

X_train_fe_sklearn[['ram','battery_power']] = X_train_fe_sklearn[['ram','battery_power']].astype('float64')
X_train_sklearn_raw = preprocessor_sklearn.fit_transform(X_train_fe_sklearn, y_train)
X_train_fe_sklearn = pd.DataFrame(X_train_sklearn_raw, columns=preprocessor_sklearn.get_feature_names_out())
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display (X_train_fe_sklearn)

,num__battery_power,num__clock_speed,num__fc,num__int_memory,num__m_dep,num__mobile_wt,num__n_cores,num__pc,num__px_height,num__px_width,num__ram,num__sc_h,num__sc_w,num__talk_time,num__screen_area,num__pixel_density,num__total_memory,num__is_high_end,cat__blue_0,cat__blue_1,cat__blue_2,cat__blue_3,cat__dual_sim_0,cat__dual_sim_1,cat__dual_sim_2,cat__dual_sim_3,cat__four_g_0,cat__four_g_1,cat__four_g_2,cat__four_g_3,cat__three_g_0,cat__three_g_1,cat__three_g_2,cat__three_g_3,cat__touch_screen_0,cat__touch_screen_1,cat__touch_screen_2,cat__touch_screen_3,cat__wifi_0,cat__wifi_1,cat__wifi_2,cat__wifi_3,quantile__battery_power,quantile__clock_speed,quantile__fc,quantile__int_memory,quantile__m_dep,quantile__mobile_wt,quantile__n_cores,quantile__pc,quantile__px_height,quantile__px_width,quantile__ram,quantile__sc_h,quantile__sc_w,quantile__talk_time,quantile__screen_area,quantile__pixel_density,quantile__total_memory,quantile__is_high_end,poly__1,poly__ram,poly__battery_power,poly__ram^2,poly__ram battery_power,poly__battery_power^2,spline__ram_sp_0,spline__ram_sp_1,spline__ram_sp_2,spline__ram_sp_3,spline__ram_sp_4
0,-0.799392,0.447049,-0.772632,-0.787867,-1.400829,0.352868,0.665738,-1.155397,-1.377002,0.436881,-0.761468,1.122914,2.376716,-1.276580,2.528916,-0.484778,-0.774012,-0.323604,0.249581,0.259611,0.262970,0.227836,0.236355,0.234747,0.260443,0.268452,0.253628,0.260076,0.232669,0.253624,0.239166,0.246942,0.265840,0.248052,0.245105,0.271201,0.238589,0.245101,0.244671,0.241390,0.249600,0.264338,0.271223,0.632633,0.292292,0.278278,0.000000,0.596096,0.692693,0.169670,0.025244,0.629129,0.279780,0.833333,0.979980,0.125125,0.975475,0.098626,0.273881,0.0,0.0,-0.761468,-0.799392,-0.846295,-0.865243,-0.852220,0.014312,0.441639,0.514965,0.029085,0.000000
1,0.267366,1.680118,-1.003974,0.813973,-0.012282,1.279294,0.665738,-0.990701,-0.091115,-0.478822,1.394557,1.598292,2.608116,-1.092544,3.198868,-0.450316,1.406985,-0.323604,0.240145,0.245070,0.245082,0.269699,0.250873,0.247369,0.264895,0.236861,0.251731,0.232664,0.277737,0.237864,0.279888,0.230057,0.243372,0.246671,0.252848,0.264246,0.233313,0.249589,0.247575,0.242727,0.245970,0.263727,0.573574,0.970470,0.000000,0.737738,0.501001,0.873373,0.692693,0.216216,0.526527,0.366867,0.901902,1.000000,0.990490,0.178178,0.993493,0.213477,0.903037,0.0,0.0,1.394557,0.267366,1.593296,1.339529,0.101249,0.000000,0.001281,0.280957,0.631571,0.086192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,0.480264,0.077129,0.152735,0.924444,-1.053692,1.475809,-0.651667,1.644426,1.481800,0.418241,1.389922,-0.065531,1.219716,0.931850,0.689832,-0.211398,1.404206,-0.323604,0.255308,0.264152,0.262400,0.218133,0.242682,0.247556,0.244318,0.265442,0.244297,0.244293,0.281148,0.230258,0.240669,0.245057,0.262630,0.251644,0.245505,0.265108,0.245516,0.243869,0.266523,0.246181,0.254671,0.232622,0.636637,0.529029,0.635636,0.762763,0.208709,0.928929,0.312312,1.000000,0.907791,0.621332,0.900967,0.467968,0.865365,0.771271,0.782282,0.616615,0.901830,0.0,0.0,1.389922,0.480264,1.585584,1.534164,0.339258,0.000000,0.001333,0.282667,0.630667,0.085333
1497,0.672779,1.556811,1.309444,-0.235508,-1.053692,0.689750,0.226603,0.820948,0.358946,-0.455521,-0.942218,-0.065531,-1.094284,-0.724472,-0.886526,0.905476,-0.945343,-0.323604,0.244232,0.242583,0.252480,0.260704,0.255764,0.250820,0.249185,0.244230,0.254829,0.256438,0.237125,0.251607,0.240184,0.244545,0.266363,0.248907,0.244452,0.237624,0.268353,0.249570,0.237416,0.243909,0.258531,0.260142,0.690763,0.934434,0.873874,0.441441,0.208709,0.699199,0.571071,0.740741,0.671424,0.372372,0.227141,0.467968,0.139640,0.295796,0.167167,0.915067,0.226402,0.0,0.0,-0.942218,0.672779,-0.946275,-0.546656,0.568150,0.027047,0.507012,0.450288,0.015652,0.000000


In [24]:
pipeline_sklearn = Pipeline(steps=[
    ('transform', preprocessor_sklearn),
    ('model', regressor)
])

model_sklearn = pipeline_sklearn.fit(X_train, y_train)

predictions = model_sklearn.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)

{'MAE': 0.16174000000000002, 'MSE': 0.0704442, 'R2': 0.9445293306119363}


In [25]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name="sklearn_regressor", experiment_id=experiment_id) as run:
   
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)

    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/12/23 19:59:34 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/12/23 19:59:34 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc

9. С использованием библиотеки mlxtend отбираем N наиболее важных признаков. N выбирается с учетом количества признаков на предыдущем шаге, ориентировочный диапазон - от 20% до 70%, после чего обучаем модель и логируем её

In [26]:
class ColumnExtractor(object):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        return X[:, self.cols]
    
    def fit(self, X, y=None):
        return self

In [27]:
sfs = SequentialFeatureSelector(
    RandomForestRegressor(random_state=43), 
    k_features=3,
    forward=True,
    floating=False,
    scoring='neg_mean_absolute_error',
    cv=2
)

sfs.fit(X_train_fe_sklearn, y_train)

sfs_cols = list(X_train_fe_sklearn.columns[list(sfs.k_feature_idx_)])
sfs_idx = list(sfs.k_feature_idx_)

print(sfs_cols)

with open('sfs_cols.txt', 'w+') as f:
    f.write(str(sfs_cols))
with open('sfs_idx.txt', 'w+') as f:
    f.write(str(sfs_idx))


sfs_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_sklearn),
    ('sfs_extractor', ColumnExtractor(sfs_idx)),
    ('model', regressor)
])

sfs_pipeline.fit(X_train, y_train)


predictions_sfs = sfs_pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions_sfs)
metrics["MSE"] = mean_squared_error(y_test, predictions_sfs)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = 'sequential_feature_selection'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    
   
    mlflow.sklearn.log_model(
        sfs_pipeline, 
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    mlflow.log_metrics(metrics)
    mlflow.log_artifact('sfs_cols.txt')
    mlflow.log_artifact('sfs_idx.txt')
    
    params = {
        'n_features_selected': len(sfs_cols),
        'forward': True,
        'floating': False,
        'scoring': 'neg_mean_absolute_error'
    }
    mlflow.log_params(params)
    
    model_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
  
    client = mlflow.tracking.MlflowClient()
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=model_version.version,
        description="Random Forest with Sequential Feature Selection"
    )

print(f"Model registered as version {model_version.version} of {REGISTRY_MODEL_NAME}")

['num__px_width', 'poly__ram battery_power', 'spline__ram_sp_3']


2025/12/23 20:01:28 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


{'MAE': 0.16212000000000001, 'MSE': 0.08960040000000001, 'R2': 0.9445293306119363}


2025/12/23 20:01:29 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Successfully registered model 'price_range_prediction_model'.
2025/12/23 20:01:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: price_range_prediction_model, version 1
Created version '1' of model 'price_range_prediction_model'.
2025/12/23 20:01:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run sequential_feature_selection at: http://127.0.0.

Model registered as version 1 of price_range_prediction_model


10. С помощью optuna настравиваем оптимальные параметры для модели, показывающей лучший результат. Обучаем модель и логируем её

In [28]:
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }
    
    regressor_trial = RandomForestRegressor(**params, random_state=44)
    pipeline_trial = Pipeline(steps=[
        ('preprocessor', preprocessor_sklearn), 
        ('sfs_extractor', ColumnExtractor(sfs_idx)),  
        ('model', regressor_trial)
    ])  
    
    pipeline_trial.fit(X_train, y_train)
    predictions = pipeline_trial.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)

    return mae

study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=30)  

best_params = study.best_params
print("Лучшие параметры:", best_params)

regressor_optimized = RandomForestRegressor(**best_params, random_state=42)
sfs_optuna_pipeline = Pipeline(steps=[  
    ('preprocessor', preprocessor_sklearn), 
    ('sfs_extractor', ColumnExtractor(sfs_idx)),  
    ('model', regressor_optimized)
])

sfs_optuna_pipeline.fit(X_train, y_train)

predictions_optuna = sfs_optuna_pipeline.predict(X_test)
metrics = {
    "MAE": mean_absolute_error(y_test, predictions_optuna),
    "MSE": mean_squared_error(y_test, predictions_optuna),
    "R2": r2_score(y_test, predictions_optuna)
}

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = 'sfs_feature_selection_optimised_Optuna'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(
        sfs_optuna_pipeline,  
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    mlflow.log_metrics(metrics)
    mlflow.log_params(best_params)
    mlflow.log_artifact('sfs_cols.txt')  
    mlflow.log_artifact('sfs_idx.txt')   
    
    model_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
    client = mlflow.tracking.MlflowClient()
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=model_version.version,
        description="Optimized Random Forest with Optuna and SFS feature selection"
    )

print(f"Model registered as version {model_version.version} of {REGISTRY_MODEL_NAME}")

[I 2025-12-23 20:04:33,152] A new study created in memory with name: no-name-d3ce108f-8912-4461-9976-3fefbe4201a5
[I 2025-12-23 20:04:33,254] Trial 0 finished with value: 0.6070497109127185 and parameters: {'max_depth': 1, 'n_estimators': 57, 'max_features': 0.5317610801556919, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.6070497109127185.
[I 2025-12-23 20:04:33,338] Trial 1 finished with value: 0.1958185009654948 and parameters: {'max_depth': 20, 'n_estimators': 25, 'max_features': 0.5054939471384295, 'min_samples_split': 10, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.1958185009654948.
[I 2025-12-23 20:04:33,427] Trial 2 finished with value: 0.20502519014333803 and parameters: {'max_depth': 9, 'n_estimators': 32, 'max_features': 0.46287690125204095, 'min_samples_split': 4, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.1958185009654948.
[I 2025-12-23 20:04:33,530] Trial 3 finished with value: 0.1834352334539739 and parameters: {'max_d

Лучшие параметры: {'max_depth': 18, 'n_estimators': 13, 'max_features': 0.9979933676529172, 'min_samples_split': 8, 'min_samples_leaf': 2}


2025/12/23 20:04:37 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Registered model 'price_range_prediction_model' already exists. Creating a new version of this model...
2025/12/23 20:04:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: price_range_prediction_model, version 2
Created version '2' of model 'price_range_prediction_model'.
2025/12/23 20:04:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run sfs_fe

Model registered as version 2 of price_range_prediction_model


11. Вычисляем среди всех экспериментов модель, показывающую наилучшее качество, обучаем её на всей выборке (а не только на train-части), логируем и ставим тег Production для последующего использования

In [34]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
best_run = runs.loc[runs['metrics.MAE'].idxmin()]
best_run_id = best_run['run_id']
best_mae = best_run['metrics.MAE']

print(f"Лучшая модель: Run ID {best_run_id}")
print(f"Лучший MAE: {best_mae}")
print(f"Метод: {best_run['tags.mlflow.runName']}")

X_full = df.drop('price_range', axis=1)
y_full = df['price_range']

print(f"Размер всей выборки: {X_full.shape}")

Лучшая модель: Run ID d67436d833194e10892ef588786b2490
Лучший MAE: 0.16174000000000002
Метод: sklearn_regressor
Размер всей выборки: (1998, 24)


In [31]:
best_run = mlflow.get_run(best_run_id)
best_params = best_run.data.params

print("Параметры лучшей модели:", best_params)

def get_param(params_dict, *keys, default, converter=int):
    for key in keys:
        value = params_dict.get(key)
        if value is not None and value != 'None':
            try:
                return converter(value)
            except (ValueError, TypeError):
                continue
    return default

final_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),  
            ('cat', TargetEncoder(), binary_features), 
            ('quantile', QuantileTransformer(), numeric_features),
            ('poly', Pipeline(steps=[
                ('poly', PolynomialFeatures(degree=2)),
                ('scale', StandardScaler())
            ]), ['ram', 'battery_power']),
            ('spline', SplineTransformer(n_knots=3), ['ram'])
        ],
        remainder='drop'
    )),
    ('sfs_extractor', ColumnExtractor(sfs_idx)),
    ('model', RandomForestRegressor(
        random_state=42,
        max_depth=get_param(best_params, 'model__max_depth', 'max_depth', default=10, converter=int),
        n_estimators=get_param(best_params, 'model__n_estimators', 'n_estimators', default=100, converter=int),
        max_features=get_param(best_params, 'model__max_features', 'max_features', default=1.0, converter=float),
        min_samples_split=get_param(best_params, 'model__min_samples_split', 'min_samples_split', default=2, converter=int),
        min_samples_leaf=get_param(best_params, 'model__min_samples_leaf', 'min_samples_leaf', default=1, converter=int)
    ))
])

final_model = final_pipeline.fit(X_full, y_full)
print("Модель успешно обучена на всех данных!")

Параметры лучшей модели: {'memory': 'None', 'steps': "[('preprocessor', ColumnTransformer(transformers=[('num', StandardScaler(),\n                                 ['battery_power', 'clock_speed', 'fc',\n                                  'int_memory', 'm_dep', 'mobile_wt', 'n_cores',\n                                  'pc', 'px_height', 'px_width', 'ram', 'sc_h',\n                                  'sc_w', 'talk_time', 'screen_area',\n                                  'pixel_density', 'total_memory',\n                                  'is_high_end']),\n                                ('bin', TargetEncoder(),\n                                 ['blue', 'dual_sim', 'four_g', 'three_g',\n                                  'touch_screen', 'wifi'])])), ('model', RandomForestRegressor(random_state=42))]", 'transform_input': 'None', 'verbose': 'False', 'preprocessor': "ColumnTransformer(transformers=[('num', StandardScaler(),\n                                 ['battery_power', 'clock_speed', 'fc

In [35]:
final_predictions = final_model.predict(X_test)

cv_scores = cross_val_score(final_model, X_full, y_full, 
                           scoring='neg_mean_absolute_error', cv=5)
final_mae_cv = -np.mean(cv_scores)

metrics = {
    "MAE": mean_absolute_error(y_test, final_predictions),
    "MSE": mean_squared_error(y_test, final_predictions),
    "R2": r2_score(y_test, final_predictions)
}

print(f"CV MAE на всех данных: {final_mae_cv:.4f}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name='Final_Production_Full_Data', experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(
        final_model,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    
    
    mlflow.log_metrics(metrics)
    
    
    mlflow.log_param("MAE_cv", final_mae_cv)
    mlflow.log_param("MAE_cv_std", np.std(cv_scores))
    
    mlflow.log_params(best_params)
    mlflow.log_param("training_data", "full_dataset")
    mlflow.log_param("training_samples", len(X_full))
    mlflow.log_param("original_best_run", best_run_id)
    mlflow.log_param("model_type", "sequential_feature_selection")
    
    production_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",  
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
    client = mlflow.tracking.MlflowClient()  
    client.set_model_version_tag(
        name=REGISTRY_MODEL_NAME,
        version=production_version.version,
        key="stage",
        value="Production"
    )
    
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=production_version.version,
        description=f"Final Production Model - Sequential Feature Selection trained on full dataset ({len(X_full)} samples)"
    )
    
    production_run_id = run.info.run_id
    print(f"Production модель зарегистрирована как версия {production_version.version}")

2025/12/23 20:32:22 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CV MAE на всех данных: 0.1556


2025/12/23 20:32:22 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.10.8, required: matplotlib==3.10.6)
 - pandas (current: 2.3.3, required: pandas==2.3.2)
 - scikit-learn (current: 1.8.0, required: scikit-learn==1.7.2)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Registered model 'price_range_prediction_model' already exists. Creating a new version of this model...
2025/12/23 20:32:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: price_range_prediction_model, version 4
Created version '4' of model 'price_range_prediction_model'.
2025/12/23 20:32:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run Final_

Production модель зарегистрирована как версия 4


In [36]:
print(f"MAE для модели обученной на всей выборке: {metrics['MAE']:.2%}")
print(f"MSE для модели обученной на всей выборке: {metrics['MSE']:.2%}")
print(f"R2 для модели обученной на всей выборке: {metrics['R2']:.2%}")

MAE для модели обученной на всей выборке: 6.53%
MSE для модели обученной на всей выборке: 1.54%
R2 для модели обученной на всей выборке: 98.79%
